In [50]:
import numpy as np
import random
import os
import math
from screeninfo import get_monitors
import time
import pandas as pd
import cv2


OUTPUT_PATH = r"C:\Users\yonir\Desktop\smooth tracking\data\third Batch"
FIELD_OF_VIEW = 90.0
xdim = int(FIELD_OF_VIEW)
ydim = int(90)


def grating_vertical(color1, color2, spatial_freq):
    pixperdeg = int(spatial_freq)
    pic = np.ones([ydim,xdim],dtype = "int8")*color1
    for x in range(int(xdim/(pixperdeg))):
        pic[:,2*x*pixperdeg:(2*x+1)*pixperdeg] = color2
    return pic

def bar_vertical(width, color, color_b, offset =0):
    offset = int((xdim/2) + offset - (width/2))
    pic = np.ones([ydim,xdim],dtype = "int8")*color_b
    pic[:,offset:offset+width] = color
    return pic
    
def checker_vertical(pixel_size=5, width_in_pixels=3, color1=0, color2=254, color_b=100, offset=0):
    squares_per_row, squares_per_col = int(width_in_pixels), int(ydim / pixel_size)
    pic = np.ones([ydim,xdim],dtype = "int8")*color_b  
    offset = int((xdim/2) + offset - (pixel_size*width_in_pixels/2))
    for i in range(squares_per_row):
        for j in range(squares_per_col):
            offsetx = (i * pixel_size) + offset
            offsety = (j * pixel_size)
            pic[offsety : offsety + pixel_size, offsetx : offsetx + pixel_size] = np.random.choice([color1, color2])
    return pic

def colored_screen(color):
    pic = np.ones([ydim,xdim],dtype = "int8")*color
    return pic

def checker_screen(pixel_size, color1, color2):
    squares_per_row, squares_per_col = int(xdim/pixel_size), int(ydim / pixel_size)
    pic = np.ones([ydim,xdim],dtype = "int8")
    for i in range(squares_per_row):
        for j in range(squares_per_col):
            offsetx = (i * pixel_size)
            offsety = (j * pixel_size)
            pic[offsety : offsety + pixel_size, offsetx : offsetx + pixel_size] = np.random.choice([color1, color2])
    return pic

def Superpose(foreground, background = None, mask = None, shift = None):
    new_indices = (np.arange(xdim) - int(shift)) % xdim
    foreground = foreground[:, new_indices]
    transparent = foreground == -1
    
    if mask is not None:
        transparent = transparent | mask

    if background is not None:
        foreground[transparent] = background[transparent]
    
    else:
        foreground[transparent] = 0
    
    return foreground

def LoopStimulus(scenes, bouncing_limits, side_duration, side_per_scene, break_duration, iteration=1, framerate=60, inverted=False, rot_offset=(0,0,0)):

    fpss = np.array([])
    fps = 0
    timer = cv2.getTickCount()
    frames=0
    print("video framerate =",framerate)
    framerate = framerate
    time_start = time.time()
    oldframe = 0

    scene_number = len(scenes)
    Abs_side_start = 0.0
    Abs_scene_start = 0.0
    Abs_loop_start = 0.0
    current_scene = 0
    current_side = 0
    current_loop = 0
    new_scene = True
    new_side = True
    new_loop = True
    in_break = True
    
    foreground = 0
    background = 0

    log = []
   
    while current_loop <= iteration:
        theoretical_elapsed_time = frames*(1/framerate)
        dt = time.time()-time_start
        now = time.time()
        pic = oldframe
        
        if (dt > theoretical_elapsed_time) | isinstance(oldframe, int):    
            frames += 1

            if new_loop:
                Abs_loop_start = now
                current_scene = 0
                new_scene = True
                new_loop = False
            
            if new_scene:
                Abs_scene_start = now
                current_side = 0
                foreground = scenes[current_scene][0]
                background = scenes[current_scene][1]
                new_scene = False
                
            scene_elapsed_time = now - Abs_scene_start - break_duration
            loop_elapsed_time = now - Abs_loop_start
            
            if scene_elapsed_time < 0:
                in_break = True
                pic = np.zeros([ydim, xdim],dtype = "uint8")

            else:
                if in_break:
                    new_side = True
                    in_break = False
                
                if new_side:
                    Abs_side_start = now
                    new_side = False
                
                side_elpased_time = now - Abs_side_start
                    
                if side_elpased_time >= side_duration:
                    log.append(("side", current_side, Abs_side_start, now, now-Abs_side_start))
                    new_side = True
                    current_side += 1
                
                if current_side == side_per_scene:
                    log.append(("scene", current_scene, Abs_scene_start, now, now-Abs_scene_start))
                    new_scene = True
                    current_scene += 1
                
                if current_scene >= scene_number:
                    log.append(("loop", current_loop, Abs_loop_start, now, now-Abs_loop_start ))
                    new_loop = True
                    current_loop += 1
                    
                if (not new_scene) & (not new_side) & (not new_loop):
                    is_side_even = - 1 + (current_side%2 != 0)*2 #give -1 or 1
                    starting_pos = -is_side_even*bouncing_limits/2  #calculate the supposed starting when the side's phase begun 
                    progress = 1-(side_duration - side_elpased_time)/side_duration #give a number between 0 and 1
                    shift = starting_pos + progress*bouncing_limits*is_side_even
                    pic = scenes[current_scene][2](foreground, background, shift)
                
        resized = cv2.cvtColor(pic.astype(np.uint8), cv2.COLOR_GRAY2RGB)
        oldframe = pic
        cv2.namedWindow('image', cv2.WND_PROP_FULLSCREEN)
        cv2.setWindowProperty('image', cv2.WND_PROP_FULLSCREEN, cv2.WINDOW_NORMAL)
        cv2.imshow('image', resized)
        tick = cv2.getTickCount()-timer
        fps = cv2.getTickFrequency()/(tick)
        timer = cv2.getTickCount()
        fpss = np.append(fpss,fps)
        key = cv2.waitKey(1)#pauses for 1ms seconds before fetching next image
        if key == 27:#if ESC is pressed, exit loop
            cv2.destroyAllWindows()
            break

    print ("mean fps " + str(np.mean(fpss)))
    df = pd.DataFrame(log, columns=["Event", "ID", "AbsoluteStart", "AbsoluteEnd", "Duration"])
    return df


half_screen = np.concatenate((np.ones([ydim, int(xdim/2)], dtype = "bool"), np.zeros([ydim, int(xdim/2)], dtype = "bool")), axis =1)

LoopStimulus(bouncing_limits = 60, side_duration = 3.0, side_per_scene = 4, break_duration =1.0, framerate = 60, iteration = 20, inverted=True, rot_offset=(0,50,0),

             #different visuals of the looping stimulus
             scenes = [
                 
                 [
                     "Black Bar on Cheker", #name of the scene
                     bar_vertical(width=15, color=0, color_b=-1), #generate foreground picture
                     checker_screen(pixel_size=10, color1=0, color2=80), #generate background picture
                     lambda f, b, s : (Superpose(foreground = f, background = b, shift = s)) #function to adapt the foreground and background with an update angular shift
                 ],
                 
                 [
                    "Black Bar on White", #name of the scene
                     bar_vertical(width=30, color=0, color_b=125),
                     None, #the background should always be specified but can be None
                     lambda f, b, s : (Superpose(foreground = f, background = b, shift = s))
                 ],
                 
                 [ # convergent and divergent scenes can be done by a background mirroring the foreground
                     "Convergent Grating", #name of the scene
                     grating_vertical(color1 = 255, color2 = 0, spatial_freq = 15),
                     None, 
                     lambda f, b, s : (Superpose(foreground = f, shift = s, mask = half_screen, background = Superpose(foreground = f, shift = -s)))
                 ]
             ])

video framerate = 60
mean fps 68.17772572375395


,Event,ID,AbsoluteStart,AbsoluteEnd,Duration
0,side,0,1.706032e+09,1.706032e+09,3.014591
1,side,1,1.706032e+09,1.706032e+09,3.009881
2,side,2,1.706032e+09,1.706032e+09,3.005637
3,side,3,1.706032e+09,1.706032e+09,3.005518
4,scene,0,1.706032e+09,1.706032e+09,13.107356
